In [ ]:
#############
## Bitcoin ##
#############

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install optuna
!pip install optuna-integration[tfkeras]
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
#######################
###### Libraries ######
#######################

import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf

from hmmlearn.hmm import GaussianHMM
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from optuna.integration import TFKerasPruningCallback
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BTC_noscale.csv')
data.drop(columns=['Crypto', 'Open', 'High', 'Low'], inplace=True)
data.head()

,Date,Close
0,2018-01-01,13535.0
1,2018-01-02,14770.0
2,2018-01-03,15057.0
3,2018-01-04,14921.0
4,2018-01-05,16828.0


In [ ]:
# -------------------------
# Step 1 - Normalisation
# -------------------------
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Close']].values.reshape(-1, 1))
input_data = scaled_data.reshape(-1)

# ---------------------------------
# Step 2 - Generate sliding windows
# ---------------------------------
train_end = int(0.65 * len(scaled_data)            ) # training ratio
val_end   = train_end + int(0.10 * len(scaled_data)) # validation ratio
window = 120                                         # window size - days of price input

train_slice = input_data[:train_end]
val_slice   = input_data[train_end:val_end]
test_slice  = input_data[val_end:]

def sequence(data, window_size):
    data = np.asarray(data).reshape(-1, 1)
    x, y = [], []
    for i in range(window_size, len(data)):
        x.append(data[i-window_size:i])
        y.append(data[i])
    return np.array(x), np.array(y)

x_train, y_train = sequence(train_slice, window)
x_val,   y_val   = sequence(val_slice,   window)
x_test,  y_test  = sequence(test_slice,  window)

n_timesteps = x_train.shape[1]
n_features  = x_train.shape[2]

# ---------------------------------
# Step 3 - Build Gaussian HMM
# ---------------------------------

# Fit Gaussian HMM with 3 states on the training input data
hmm = GaussianHMM(n_components=3, n_iter=200, random_state=42)
hmm.fit(train_slice.reshape(-1,1))

# Build train, val, test arrays with the last indices in each LSTM sequence (above)
# The aim is to align the HMM input data to the LSTM sequences of 120-day input
# To avoid data leakage (into the future)
window = 120
train_last_idx = np.arange(window-1, train_end-1)
val_last_idx   = np.arange(train_end + window-1, val_end-1)
test_last_idx  = np.arange(val_end   + window-1, len(input_data)-1)


# -----------------------------------------
# Step 5 — Compute posterior probabilities
# -----------------------------------------

# Build the Causal HMM with the input data and train/val/test indices from above
# The probability of the next point is only based on the past datapoints
# Compute the posterior probability for each price at time t+1 for input 0...t (t=window-1)
# The post_probability of each of the 3 states is returned for all 3 sets and ready for use in training the model
def causal_hmm(price_series, hmm, index):
    post_probabilities = np.zeros((len(index), hmm.n_components))
    for i, t in enumerate(index):
        post_probabilities[i] = hmm.predict_proba(price_series[:t+1].reshape(-1,1))[-1]  # last row = uses data ≤ t
    return post_probabilities

H_train = causal_hmm(input_data, hmm, train_last_idx)
H_val   = causal_hmm(input_data, hmm, val_last_idx)
H_test  = causal_hmm(input_data, hmm, test_last_idx)


In [ ]:
def objective(trial):
    # --------------------
    # Hyperparameters
    # --------------------
    lstm_units1 = trial.suggest_categorical('lstm_units1', [16, 32, 64, 128])
    lstm_units2 = trial.suggest_categorical('lstm_units2', [8, 16, 32, 64])
    lstm_units3 = trial.suggest_categorical('lstm_units3', [2, 4, 8, 16])
    dense_units = trial.suggest_categorical('dense_units', [4, 8, 32, 64])
    dense_units2 = trial.suggest_categorical('dense_units2', [4, 8, 32, 64])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_choice = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 30, 200, step=10)

    # --------------------
    # Optimizer
    # --------------------
    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    # --------------------
    # LSTM branch
    # --------------------
    lstm_input = Input(shape=(n_timesteps, n_features))
    x = LSTM(lstm_units1, return_sequences=True, activation=activation)(lstm_input)
    x = Dropout(dropout_rate)(x)
    x = LSTM(lstm_units2, return_sequences=True, activation=activation)(x)
    x = LSTM(lstm_units3, return_sequences=False, activation=activation)(x)


    # ----------------------------------------
    # HMM branch (precomputed states as input)
    # ----------------------------------------
    # Input = one probability for each of 3 states
    hmm_input = Input(shape=(3,))
    h         = Dense(dense_units, activation=activation)(hmm_input)

    # --------------------
    # Concatenate
    # --------------------
    combined = Concatenate()([x, h])
    combined = Dense(dense_units2, activation=activation)(combined)
    output = Dense(1, activation='linear')(combined)

    # --------------------
    # Build & compile
    # --------------------
    model = Model(inputs=[lstm_input, hmm_input], outputs=output)
    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=['mae', RootMeanSquaredError(), 'mape'])

    # --------------------
    # Callbacks
    # --------------------
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    pruning_cb = TFKerasPruningCallback(trial, 'val_loss')

    # --------------------
    # Training
    # --------------------
    history = model.fit(
        [x_train, H_train], y_train,
        validation_data=([x_val, H_val], y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop, pruning_cb]
    )

    # --------------------
    # Objective metric
    # --------------------
    val_loss = min(history.history['val_loss'])
    return val_loss


In [ ]:
#### Run optuna ####
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2025-09-27 03:01:57,377] A new study created in memory with name: no-name-1e37561a-3f3a-4545-9aeb-0471e81c23b6
[I 2025-09-27 03:03:21,589] Trial 0 finished with value: 0.002558342181146145 and parameters: {'lstm_units1': 128, 'lstm_units2': 8, 'lstm_units3': 8, 'dense_units': 4, 'dense_units2': 4, 'dropout_rate': 0.36417395008752473, 'activation': 'tanh', 'optimizer': 'adam', 'learning_rate': 0.00040301498485002487, 'batch_size': 16, 'epochs': 150}. Best is trial 0 with value: 0.002558342181146145.
[I 2025-09-27 03:04:26,932] Trial 1 finished with value: 0.0009710186859592795 and parameters: {'lstm_units1': 32, 'lstm_units2': 32, 'lstm_units3': 16, 'dense_units': 4, 'dense_units2': 32, 'dropout_rate': 0.2456353443452543, 'activation': 'tanh', 'optimizer': 'adam', 'learning_rate': 0.0008373976142963422, 'batch_size': 16, 'epochs': 50}. Best is trial 1 with value: 0.0009710186859592795.
[I 2025-09-27 03:06:07,365] Trial 2 finished with value: 0.003869310487061739 and parameters: {'lst

In [ ]:
print("Best trial:")
trial = study.best_trial

print(f"  Loss: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Loss: 0.0008035983191803098
  Params: 
    lstm_units1: 128
    lstm_units2: 8
    lstm_units3: 8
    dense_units: 4
    dense_units2: 64
    dropout_rate: 0.09246768282596343
    activation: tanh
    optimizer: adam
    learning_rate: 0.0027441467785781193
    batch_size: 16
    epochs: 140


In [ ]:
from google.colab import files

results = study.trials_dataframe()
results.to_csv('optuna_lstm_hmm_results.csv', index=False)

files.download('optuna_lstm_hmm_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>